In [ ]:
!pip install -q langchain openai
!pip install -q langchain-experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.callbacks import get_openai_callback

In [ ]:
!pip install "gdown==4.6"

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [ ]:
!gdown --id 1zYVU1kB_eVCSWHNUhWrYVDF-v2djxYM5

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zYVU1kB_eVCSWHNUhWrYVDF-v2djxYM5
To: /content/Validation_Dataset_Question.zip
100% 10.7M/10.7M [00:00<00:00, 71.6MB/s]


In [ ]:
!unzip Validation_Dataset_Question.zip

Archive:  Validation_Dataset_Question.zip
  inflating: Validation_Dataset_Question/code_validate_dataset.csv  
   creating: Validation_Dataset_Question/Val_Data/
  inflating: Validation_Dataset_Question/Val_Data/Airline Dataset.csv  
  inflating: Validation_Dataset_Question/Val_Data/AnalyticsVidiya.csv  
  inflating: Validation_Dataset_Question/Val_Data/BigMart.csv  
  inflating: Validation_Dataset_Question/Val_Data/customer_shopping_data.csv  
  inflating: Validation_Dataset_Question/Val_Data/MEN_SHOES.csv  


In [ ]:
from google.colab import userdata
openai_key = userdata.get('openai_api')

In [ ]:
df = pd.read_csv("Validation_Dataset_Question/Val_Data/Airline Dataset.csv")

In [ ]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", api_key=openai_key),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [ ]:
df.head()

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Departure Date,Arrival Airport,Pilot Name,Flight Status
0,10856,Edithe,Leggis,Female,62,Japan,Coldfoot Airport,US,United States,NAM,North America,2022-06-28,CXF,Edithe Leggis,On Time
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA,Canada,NAM,North America,2022-12-26,YCO,Elwood Catt,On Time
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR,France,EU,Europe,2022-01-18,GNB,Darby Felgate,On Time
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA,Canada,NAM,North America,2022-09-16,YND,Dominica Pyle,Delayed
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US,United States,NAM,North America,2022-02-25,SEE,Bay Pencost,On Time


In [ ]:
global process
def process(df):
 # Convert the date to a datetime object
 df['Departure Date'] = pd.to_datetime(df['Departure Date'], format='%m/%d/%Y')

 # Check if 'US' is present in the 'Airport Country Code' column
 us_airports = df[df['Airport Country Code'] == 'US']

 # Filter the dataframe for rows where the departure date is 2022-01-01
 january_1_2022 = pd.Timestamp('2022-01-01')
 arrivals_on_january_1 = us_airports[us_airports['Departure Date'] == '1/1/2022']

 # Count the number of rows
 number_of_arrivals = arrivals_on_january_1.shape[0]

 return f"{number_of_arrivals} passengers arrived in a US airport on 2022-01-01."

In [ ]:
process(df)

'66 passengers arrived in a US airport on 2022-01-01.'

In [ ]:
df[(df['Airport Country Code'] == 'US') & (df['Departure Date'] == '1/1/2022')].shape[0]

66

In [ ]:
df['Arrival Airport'].unique()

array(['CXF', 'YCO', 'GNB', ..., 'CXM', 'ROR', 'HPN'], dtype=object)

In [ ]:
print(agent.run("in US airport 2022/1/1 how many passengers arrived?"))



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Airport Country Code'] == 'US') & (df['Arrival Airport'] == 'US') & (df['Departure Date'] == '1/1/2022')].shape[0]"}`


0In US airports on January 1, 2022, no passengers arrived according to the given dataframe.

> Finished chain.
In US airports on January 1, 2022, no passengers arrived according to the given dataframe.


In [ ]:
with get_openai_callback() as cb:
  result = agent.run("how mnay Passenger names who are aged between 30 to 60?")
  print(cb)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "len(df[(df['Age'] >= 30) & (df['Age'] <= 60)])"}`


34009There are 34,009 passengers who are aged between 30 to 60.

> Finished chain.
Tokens Used: 1308
	Prompt Tokens: 1254
	Completion Tokens: 54
Successful Requests: 2
Total Cost (USD): $0.001989


In [ ]:
with get_openai_callback() as cb:
  result = agent.run("how mnay Passenger names who are aged between 30 to 60?")
  print(cb)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['Age'] >= 30) & (df['Age'] <= 60)].shape[0]"}`


34009There are 34,009 passengers aged between 30 to 60.

> Finished chain.
Tokens Used: 1310
	Prompt Tokens: 1256
	Completion Tokens: 54
Successful Requests: 2
Total Cost (USD): $0.04092
